# Segment, localise and track


In [9]:
from macrohet import dataio, tile, notify
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

model_path = '/mnt/DATA/macrohet/upstream_development/segmentation/cellpose_training/models/macrohet_seg'
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# # defining cellpose model to use
# model = models.Cellpose(
#                         gpu=True, 
#                         model_type='cyto', 
#                         net_avg=True, 
#                         device=torch.device('cuda')
#                         )

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.1 GB


### Define functions to tidy up main block of code

In [2]:
def segment(frame, model = model, channels = [0,0], diameter = 0,#325
            min_size = 0 #2500
           ):
    
#     masks, flows, styles, diams = model.eval(frame, 
#                                              channels = channels, 
#                                              diameter = diameter, 
#                                              min_size = min_size, 
#                                              )
    masks, flows, styles = model.eval(frame, 
                                      channels = channels, 
                                      diameter = diameter, 
                                      min_size = min_size, 
                                      )
    return masks

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

scale_factor = 1/5.04

def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects

# define config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/btrack/models/particle_config_pnassign.json'

def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'

In [4]:
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [5]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Segment, localise and track

In [6]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')

In [7]:
import time

In [10]:
time.sleep(60*90)

In [11]:
notify.send_sms('Timer done, continuing to segment')

In [ ]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    if (row, column) == (3, 4):
        continue
    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # segment images from gfp channel only
    masks = np.stack([segment(frame) 
                      for frame in tqdm(images[:,0,...], 
                                        desc = 'Segmenting')])

    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 20)

    # save out 
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model2/{row, column}.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
#             writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
    print(f"Checking time indices: {row, column, tracks[0]['t']}")
    notify.send_sms(f"Position {row, column} saved out (model2), {tracks[0]['t']}")

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/05/16 08:43:19 PM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████████████| 75/75 [15:28<00:00, 12.38s/it]
[INFO][2023/05/16 08:58:47 PM] Objects are of type: <class 'dict'>
[INFO][2023/05/16 08:58:48 PM] ...Found 46289 objects in 75 frames.
[INFO][2023/05/16 08:58:48 PM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/16 08:58:48 PM] Starting BayesianTracker session
[INFO][2023/05/16 08:58:48 PM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/16 08:58:48 PM] Objects are of type: <class 'list'>
[INFO][2023/05/16 08:58:49 PM] Starting tracking... 
[INFO][2023/05/16 08:58:49 PM] Update using: ['MOTION', 'VISUAL']
[INFO][2023/05/16 08:58:49 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/05/16 08:58:59 PM]  - Timing (Bayesian updates: 150.46ms, Linking: 4.63ms)
[INFO][2023/05/16 08:58:59 PM]  - Probabilities (Li

GLPK Integer Optimizer 5.0
15792 rows, 14084 columns, 20272 non-zeros
14084 integer variables, all of which are binary
Preprocessing...
7896 rows, 14084 columns, 20272 non-zeros
14084 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7896
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
7896 rows, 14084 columns, 20272 non-zeros
*     0: obj =   4.018248762e+04 inf =   0.000e+00 (3819)
Perturbing LP to avoid stalling [1560]...
Removing LP perturbation [3786]...
*  3786: obj =   1.811186108e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3786: mip =     not found yet >=              -inf        (1; 0)
+  3786: >>>>>   1.811186108e+04 >=   1.811186108e+04   0.0% (1; 0)
+  3786: mip =   1.811186108e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTI

[INFO][2023/05/16 08:59:17 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model2/(3, 5).h5...
[INFO][2023/05/16 08:59:18 PM] Writing objects/obj_type_1
[INFO][2023/05/16 08:59:18 PM] Writing labels/obj_type_1
[INFO][2023/05/16 08:59:18 PM] Loading objects/obj_type_1 (42040, 5) (42040 filtered: None)
[INFO][2023/05/16 08:59:18 PM] Writing properties/obj_type_1/area (42040,)
[INFO][2023/05/16 08:59:18 PM] Writing properties/obj_type_1/major_axis_length (42040,)
[INFO][2023/05/16 08:59:18 PM] Writing properties/obj_type_1/minor_axis_length (42040,)
[INFO][2023/05/16 08:59:18 PM] Writing properties/obj_type_1/orientation (42040,)
[INFO][2023/05/16 08:59:18 PM] Writing properties/obj_type_1/mean_intensity (42040, 2)
[INFO][2023/05/16 08:59:18 PM] Writing tracks/obj_type_1
[INFO][2023/05/16 08:59:18 PM] Writing dummies/obj_type_1
[INFO][2023/05/16 08:59:18 PM] Writing LBEP/obj_type_1
[INFO][2023/05/16 08:59:18 PM] Writing fates/obj_type_1
[INFO][2023/05/16 08:59:54 PM] Closing 

...

[INFO][2023/05/17 07:11:49 AM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████████████| 75/75 [15:44<00:00, 12.60s/it]
[INFO][2023/05/17 07:27:34 AM] Objects are of type: <class 'dict'>
[INFO][2023/05/17 07:27:34 AM] ...Found 53503 objects in 75 frames.
[INFO][2023/05/17 07:27:34 AM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/05/17 07:27:34 AM] Starting BayesianTracker session
[INFO][2023/05/17 07:27:34 AM] Loading configuration file: /home/dayn/analysis/btrack/models/particle_config_pnassign.json
[INFO][2023/05/17 07:27:34 AM] Objects are of type: <class 'list'>
[INFO][2023/05/17 07:27:35 AM] Starting tracking... 
[INFO][2023/05/17 07:27:35 AM] Update using: ['MOTION', 'VISUAL']
[INFO][2023/05/17 07:27:36 AM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/05/17 07:27:51 AM]  - Timing (Bayesian updates: 184.40ms, Linking: 5.64ms)
[INFO][2023/05/17 07:27:51 AM]  - Probabilities (Li

GLPK Integer Optimizer 5.0
18756 rows, 16964 columns, 24550 non-zeros
16964 integer variables, all of which are binary
Preprocessing...
9378 rows, 16964 columns, 24550 non-zeros
16964 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 9378
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
9378 rows, 16964 columns, 24550 non-zeros
*     0: obj =   4.718167253e+04 inf =   0.000e+00 (4774)
Perturbing LP to avoid stalling [1696]...
Removing LP perturbation [4796]...
*  4796: obj =   2.067268152e+04 inf =   0.000e+00 (0) 2
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4796: mip =     not found yet >=              -inf        (1; 0)
+  4796: >>>>>   2.067268152e+04 >=   2.067268152e+04   0.0% (1; 0)
+  4796: mip =   2.067268152e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTI

[INFO][2023/05/17 07:28:18 AM] Ending BayesianTracker session
[INFO][2023/05/17 07:28:18 AM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model2/(3, 8).h5...
[INFO][2023/05/17 07:28:19 AM] Writing objects/obj_type_1
[INFO][2023/05/17 07:28:19 AM] Writing labels/obj_type_1
[INFO][2023/05/17 07:28:19 AM] Loading objects/obj_type_1 (48427, 5) (48427 filtered: None)
[INFO][2023/05/17 07:28:20 AM] Writing properties/obj_type_1/area (48427,)
[INFO][2023/05/17 07:28:20 AM] Writing properties/obj_type_1/major_axis_length (48427,)
[INFO][2023/05/17 07:28:20 AM] Writing properties/obj_type_1/minor_axis_length (48427,)
[INFO][2023/05/17 07:28:20 AM] Writing properties/obj_type_1/orientation (48427,)
[INFO][2023/05/17 07:28:20 AM] Writing properties/obj_type_1/mean_intensity (48427, 2)
[INFO][2023/05/17 07:28:20 AM] Writing tracks/obj_type_1
[INFO][2023/05/17 07:28:20 AM] Writing dummies/obj_type_1
[INFO][2023/05/17 07:28:20 AM] Writing LBEP/obj_type_1
[INFO][2023/05/17 07:28:20 AM] Wr

Checking time indices: (3, 8, [0, 1, 2, 3, 4, 5, 6])


Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

In [ ]:
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    if (row, column) != (3, 4):
        continue
    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # segment images from gfp channel only
    masks = np.stack([segment(frame) 
                      for frame in tqdm(images[:,0,...], 
                                        desc = 'Segmenting')])


    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 20)

    # save out 
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
#             writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)
    print(f"Checking time indices: {row, column, tracks[0]['t']}")
    notify.send_sms(f"Position {row, column} saved out (model2), {tracks[0]['t']}")